# Installation

In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [3]:
import os, sys
from os.path import isfile, join
import numpy as np
from PIL import Image

os.chdir("/content/")
!ls
!mkdir -p drive
!google-drive-ocamlfuse drive
!pip install -q keras
!pip install -q opencv-python
!apt install -y libsm6 libxext6
!pip install -q keras

os.chdir("/content/drive/Colab/Portable/")
# print "Back to current dir:"
# !ls

datalab  drive
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsm6 is already the newest version (2:1.2.2-1).
libxext6 is already the newest version (2:1.3.3-1).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


OSError: ignored

#1. Prepare data

In [0]:
PATH_INPUT = "input/"
PATH_OUTPUT = "output/"

original_imgs_test = PATH_INPUT

model_path = "model/weights.hdf5"
architechture_path = "model/architecture.json"

channels = 3
# height = 1860
# width = 1328

height = 2336
width = 1696


# dimension of the patches
patch_height = 32
patch_width = 32

#the stride in case output with average
stride_height = 20
stride_width = 20
assert (stride_height < patch_height and stride_width < patch_width)

# PATH_TEMP = "temp/"
# temp_file = "data.hdf5"
# dataset_path = PATH_TEMP

In [3]:
import h5py
import numpy as np
from PIL import Image

from os import listdir
from os.path import isfile, join

#Python
import numpy as np
# import ConfigParser

#Keras
from keras.models import model_from_json
from keras.models import Model

import sys
sys.path.insert(0, './lib/')
# help_functions.py
from help_functions import load_hdf5, write_hdf5, rgb2gray, group_images, visualize, masks_Unet, pred_to_imgs

# extract_patches.py
from extract_patches import recompone
from extract_patches import recompone_overlap
from extract_patches import paint_border
from extract_patches import paint_border_overlap
from extract_patches import extract_ordered_overlap
from extract_patches import kill_border
from extract_patches import pred_not_only_FOV
from extract_patches import get_data_testing
from extract_patches import get_data_testing_overlap

# pre_processing.py
from pre_processing import my_PreProc

def write_hdf5(arr,outfile):
    with h5py.File(outfile,"w") as f:
        f.create_dataset("image", data=arr, dtype=arr.dtype)

def get_datasets(imgs_dir):
    
    Nimgs = len([f for f in listdir(imgs_dir) if isfile(join(imgs_dir, f))])
    imgs = np.empty((Nimgs,height,width,channels))
    for path, subdirs, files in os.walk(imgs_dir): #list all files, directories in the path
        for i in range(len(files)):
            print ("image: " +files[i])
            
            ### write some lines of code to:
            ### - check if image's size is different to requirement, then resize and save to output folder
            ### - else: make a copy to output folder
            
            img = Image.open(imgs_dir+files[i])
            imgs[i] = np.asarray(img)    
    imgs = np.transpose(imgs,(0,3,1,2))
    assert(imgs.shape == (Nimgs,channels,height,width))
    return imgs, files

def get_data(imgs_test, patch_height, patch_width, stride_height, stride_width):
    test_imgs_original = imgs_test
    test_imgs = my_PreProc(test_imgs_original)
    test_imgs = test_imgs[:,:,:,:]
    test_imgs = paint_border_overlap(test_imgs, patch_height, patch_width, stride_height, stride_width)

#     print "\ntest images shape:"
#     print test_imgs.shape
#     print "range (min-max): " +str(np.min(test_imgs)) +' - '+str(np.max(test_imgs))
   
    #extract the TEST patches from the full images
    patches_imgs_test = extract_ordered_overlap(test_imgs,patch_height,patch_width,stride_height,stride_width)

#     print "\ntest PATCHES images shape:"
#     print patches_imgs_test.shape
#     print "range (min-max): " +str(np.min(patches_imgs_test)) +' - '+str(np.max(patches_imgs_test))
    
    return patches_imgs_test, test_imgs.shape[2], test_imgs.shape[3]
  
def createDir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    else:
        print directory + " already exists"

Using TensorFlow backend.


#2. Predictions

In [4]:
#load testing datasets
imgs_test, file_names = get_datasets(original_imgs_test)
print ("test datasets loaded")
print(imgs_test.shape)
# write_hdf5(imgs_test,dataset_path + temp_file)

#original test images
test_imgs_orig = imgs_test
full_img_height = test_imgs_orig.shape[2]
full_img_width = test_imgs_orig.shape[3]

#Images to patches:
patches_imgs_test = None
new_height = None
new_width = None
masks_test  = None
patches_masks_test = None
# average_mode = True
# if average_mode == True:
patches_imgs_test, new_height, new_width = get_data(
    imgs_test = imgs_test,
    patch_height = patch_height,
    patch_width = patch_width,
    stride_height = stride_height,
    stride_width = stride_width
)

#Load model
model = model_from_json(open(architechture_path).read())
model.load_weights(model_path)

#Calculate the predictions
predictions = model.predict(patches_imgs_test, batch_size=32, verbose=2)

# print "predicted images size :"
# print predictions.shape

#===== Convert the prediction arrays in corresponding images
pred_patches = pred_to_imgs(predictions, patch_height, patch_width, "original")

pred_imgs = None

# average_mode = True
# if average_mode == True:
pred_imgs = recompone_overlap(pred_patches, new_height, new_width, stride_height, stride_width)# predictions
# else:
#     pred_imgs = recompone(pred_patches,13,12)


pred_imgs = pred_imgs[:,:,0:full_img_height,0:full_img_width]
# print "pred imgs shape: " +str(pred_imgs.shape)

assert(len(file_names) == pred_imgs.shape[0])

N_predicted = pred_imgs.shape[0]
# group = 1

# Save predictions to files
for i in range(int(N_predicted)):
    pred_stripe = group_images(pred_imgs[i:i+1,:,:,:],1)
    file_name =  file_names[i]
    visualize(pred_stripe, "output/" + file_name[0:len(file_name)-4] + "_pred")

print "All done! Please check the output folder"

image: 26_4_12_TW10001 (c31071e6).jpg
image: 26_4_12_TW40001 (fae95c4d).jpg
image: 26_5_12_TB1_00_300001 (77b69c52).jpg
image: 26_5_12_TB1_30_900001 (5093daa9).jpg
image: 26_5_12_TB1_60_900001 (88abbddd).jpg
image: 26_5_12_TB2_00_300001 (94f4a4a0).jpg
image: 26_5_12_TB2_30_600001 (0be70fee).jpg
image: 26_5_12_TB2_60_900001 (0418a72b).jpg
image: 26_5_12_TB3_00_300001 (e590f112).jpg
test datasets loaded
(9, 3, 2336, 1696)

the side H is not compatible with the selected stride of 20
img_h 2336, patch_h 32, stride_h 20
(img_h - patch_h) MOD stride_h: 4
So the H dim will be padded with additional 16 pixels
the side W is not compatible with the selected stride of 20
img_w 1696, patch_w 32, stride_w 20
(img_w - patch_w) MOD stride_w: 4
So the W dim will be padded with additional 16 pixels
new full images shape: 
(9, 3, 2352, 1712)
Number of patches on h : 117
Number of patches on w : 85
number of patches per image: 9945, totally for this dataset: 89505
N_patches_h: 117
N_patches_w: 85
N_patch